<a href="https://colab.research.google.com/github/JuanGranados09/SE-ALES-Y-SISTEMAS/blob/main/Implementacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pytube


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 5.6 MB/s eta 0:00:00


In [4]:
# Importar las librerías necesarias
from pytube import YouTube
import librosa
import numpy as np
from scipy import signal
import joblib  # Asumiendo que el modelo está guardado con joblib

# Cargar el modelo entrenado (ajusta el nombre del archivo según corresponda)
modelo = joblib.load('tu_modelo_entrenado.pkl')

# Función para descargar el audio de YouTube
def descargar_audio(link_youtube, salida='audio.wav'):
    yt = YouTube(link_youtube)
    audio_stream = yt.streams.filter(only_audio=True).first()
    archivo_audio = audio_stream.download(filename=salida)
    return archivo_audio

# Función para preprocesar el audio (modulación, filtrado, etc.)
def preprocesar_audio(archivo_audio):
    # Cargar el audio
    y, sr = librosa.load(archivo_audio, sr=None)

    # Normalizar el audio
    y_normalizado = librosa.util.normalize(y)

    # Aplicar un filtro básico (ejemplo: paso bajo)
    b, a = signal.butter(5, 1000/(0.5*sr), btype='low')  # Filtro paso bajo a 1kHz
    y_filtrado = signal.filtfilt(b, a, y_normalizado)

    # Extraer características del audio (mel espectrograma, MFCC, etc.)
    mfccs = librosa.feature.mfcc(y=y_filtrado, sr=sr, n_mfcc=13)

    # Aplanar las características para pasar al modelo
    caracteristicas = np.mean(mfccs.T, axis=0)

    return caracteristicas

# Función para predecir el género
def predecir_genero(caracteristicas):
    genero_predicho = modelo.predict([caracteristicas])
    if genero_predicho == 0:
        return "Reguetón"
    else:
        return "Metal"

# Función principal para realizar todo el flujo
def detectar_genero(link_youtube):
    # 1. Descargar el audio de YouTube
    archivo_audio = descargar_audio(link_youtube)

    # 2. Preprocesar el audio
    caracteristicas = preprocesar_audio(archivo_audio)

    # 3. Predecir el género
    genero = predecir_genero(caracteristicas)

    # 4. Mostrar el resultado
    print(f"El género de la canción es: {genero}")

# Ejemplo de uso
link_youtube = input("Ingresa el link de YouTube de la canción: ")
detectar_genero(link_youtube)


FileNotFoundError: [Errno 2] No such file or directory: '/ruta/a/tu_modelo/Detector_de_Genero.pkl'